In [1]:
# !pip install datasets

https://calvinfeng.gitbook.io/machine-learning-notebook/supervised-learning/recurrent-neural-network/long_short_term_memory

In [2]:
import os
import random
import numpy as np

# import nltk
import gensim.downloader as api

import torch
import torch.nn as nn
import torch.nn.functional as F
# import datasets

In [ ]:
# Зафиксируем seed для воспроизводимости

def seed_everything(seed):
    random.seed(seed) # Фиксируем генератор случайных чисел
    os.environ['PYTHONHASHSEED'] = str(seed) # Фиксируем заполнения хешей
    np.random.seed(seed) # Фиксируем генератор случайных чисел numpy
    torch.manual_seed(seed) # Фиксируем генератор случайных чисел pytorch
    torch.cuda.manual_seed(seed) # Фиксируем генератор случайных чисел для GPU
    torch.backends.cudnn.deterministic = True # Выбираем только детерминированные алгоритмы (для сверток)
    torch.backends.cudnn.benchmark = False # Фиксируем алгоритм вычисления сверток

In [ ]:
seed_everything(42)

# Дешифрация текста

Представьте, что вам даны сообщения зашифрованные с помощью шифра Цезаря, являющимся одним из самых простых шифров в криптографии.

Шифр Цезаря работает следующим образом: каждая буква исходного алфавита сдвигается на K символов вправо.

Пусть нам дано сообщение: message="RNN IS NOT AI", тогда наше шифрование выполняющиеся по правилу f, с K=2, даст нам результат:
f(message, K) = TPPAKUAPQVACK

Для удобства можно взять символы только одного регистра в нашей имплементации, и сказать, что все буквы НЕ ОТНОСЯЩИЕСЯ к английскому алфавиту будут отмечены как прочерк "-".

In [ ]:
# Определим ключ и словарь
key = 2
vocab = [char for char in ' -ABCDEFGHIJKLMNOPQRSTUVWXYZ!?']

In [ ]:
# Напишем функцию, которая делает шифр
def encrypt(text, key):
    """"Возвращает зашифрованную форму 'text'."""
    indexes = [vocab.index(char) for char in text] # для каджого символя из 'text' получаем индекс из словаря
    encrypted_indexes = [(idx + key) % len(vocab) for idx in indexes] # каждому индексу мы получаем новый зашифрованный индекс
    # print(indexes)
    # print(encrypted_indexes)
    encrypted_chars = [vocab[idx] for idx in encrypted_indexes] # получаем новую фразу
    encrypted = ''.join(encrypted_chars)
    return encrypted

print(encrypt('RNN IS NOT AI', key))

TPPAKUAPQVACK


In [ ]:
num_examples = 256 # размер датасета
seq_len = 18 # максимальная длина строки

In [ ]:
def encrypted_dataset(dataset_len, k):
    """
    Return: List(Tuple(Tensor encrypted, Tensor source))
    """
    dataset = []
    for x in range(dataset_len):
        random_message  = ''.join([random.choice(vocab) for x in range(seq_len)])
        encrypt_random_message = encrypt(''.join(random_message), k)
        src = [vocab.index(x) for x in random_message]
        tgt = [vocab.index(x) for x in encrypt_random_message]
        dataset.append([torch.tensor(tgt), torch.tensor(src)])
    return dataset

In [ ]:
class Decipher(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim,
                rnn_type='simple'):
        """
        :params: int vocab_size
        :params: int embedding_dim
        :params
        """
        super(Decipher, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        if rnn_type == 'simple':
            self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers = 2)

        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.initial_hidden = torch.zeros(2, 1, hidden_dim)


    def forward(self, cipher):
        # CHECK INPUT SIZE
        # Unsqueeze 1 dimension for batches
        embd_x = self.embed(cipher).unsqueeze(1)
        out_rnn, hidden = self.rnn(embd_x, self.initial_hidden)
        # Apply the affine transform and transpose output in appropriate way
        # because you want to get the softmax on vocabulary dimension
        # in order to get probability of every letter
        return self.fc(out_rnn).transpose(1, 2)

In [ ]:
# Определим параметры нашей модели
embedding_dim = 5
hidden_dim = 10
vocab_size = len(vocab)
lr = 1e-3

criterion = nn.CrossEntropyLoss()

# Инициализируйте модель
model = Decipher(vocab_size, embedding_dim, hidden_dim)

# Инициализируйте оптимизатор: рекомендуется Adam
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)



In [ ]:
num_epochs = 10
k = 10 # смещение
for x in range(num_epochs):
    print('Epoch: {:2.0f}'.format(x), end=', ')
    for encrypted, original in encrypted_dataset(num_examples, k):

        scores = model(encrypted)
        original = original.unsqueeze(1)
        # Calculate loss
        loss = criterion(scores, original)
        # Zero grads
        optimizer.zero_grad()
        # Backpropagate
        loss.backward()
        # Update weights
        optimizer.step()
    print('Loss: {:6.4f}'.format(loss.item()), end=',  ')

    with torch.no_grad():
        matches, total = 0, 0
        for encrypted, original in encrypted_dataset(num_examples, k):
            # Compute a softmax over the outputs
            predictions = F.softmax(model(encrypted), 1)
            # Choose the character with the maximum probability (greedy decoding)
            _, batch_out = predictions.max(dim=1)
            # Remove batch
            batch_out = batch_out.squeeze(1)
            # Calculate accuracy
            matches += torch.eq(batch_out, original).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Epoch:  0, Loss: 2.7445,  Accuracy: 31.34%
Epoch:  1, Loss: 1.8754,  Accuracy: 70.90%
Epoch:  2, Loss: 1.2083,  Accuracy: 90.17%
Epoch:  3, Loss: 0.8470,  Accuracy: 97.24%
Epoch:  4, Loss: 0.6072,  Accuracy: 99.93%
Epoch:  5, Loss: 0.4411,  Accuracy: 100.00%
Epoch:  6, Loss: 0.3088,  Accuracy: 100.00%
Epoch:  7, Loss: 0.3093,  Accuracy: 100.00%
Epoch:  8, Loss: 0.2334,  Accuracy: 100.00%
Epoch:  9, Loss: 0.1643,  Accuracy: 100.00%


In [ ]:
source = 'HELLO WORLD'
# coding
tgt = [vocab.index(x) for x in encrypt(''.join(source), k)]
# decoding
_, batch_out = F.softmax(model(torch.tensor(tgt)), 1).max(dim=1)

print(f'Source: {source}')
print(f'Encrypted: {encrypted}')
print(f'Predictions: {"".join([vocab[i] for i in batch_out])}')

Source: HELLO WORLD
Encrypted: tensor([ 7,  9, 13,  6,  6,  1, 20, 29, 12, 12, 25, 20,  0, 16, 17, 13, 24, 11])
Predictions: HELLO WORLD
